In [1]:
import tensorflow as tf
import tensorflow_hub as hub
from matplotlib import pyplot as plt
import cv2
import numpy as np

In [2]:
# Optional if you are using a GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
model = hub.load("https://tfhub.dev/google/movenet/singlepose/thunder/4")
movenet = model.signatures['serving_default']

In [4]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (255,255,255), -1) 

In [5]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [6]:
def draw_connections(frame, keypoints, edges, confidence_threshold, color):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)

In [7]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [8]:
cap = cv2.VideoCapture('Videos/squat.mp4')
stage = None
state = None
while cap.isOpened():
    ret, frame = cap.read()
    
    # Reshape image
    img = frame.copy()
    img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0),256,256)
    input_image = tf.cast(img, dtype=tf.int32)
    
    # Setup input and output 
    outputs = movenet(input_image)
    
    # Make predictions 
    keypoints_with_scores = outputs['output_0']
    
    # Calculate angles
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints_with_scores, [y,x,1]))
    try:
        shoulder = [shaped[5][1], shaped[5][0]]
        hip = [shaped[11][1], shaped[11][0]]
        knee = [shaped[13][1], shaped[13][0]]
        ankle = [shaped[15][1], shaped[15][0]]

        
        angle = calculate_angle(shoulder, hip, knee)
        cv2.putText(frame, str(angle),(int(hip[0]), int(hip[1])), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)

        angle2 = calculate_angle(hip, knee, ankle)
        cv2.putText(frame, str(angle2),(int(knee[0]), int(knee[1])), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
        #squat logic
        if ((angle > 40) & (angle < 55)) | ((angle2 > 50) & (angle2 < 55)):
            stage = 'Good Squat'
            state = 2
        elif (angle < 40) | (angle2 < 50):
            stage = 'Over Squat'
            state = 3
        elif (angle > 55) & (angle < 100):
            stage = 'Squat more'
            state = 1
        else:
            stage = 'Not squatted'
            state = 0
            
    except:
        pass
    
    # Stage data
    if state == 2:
        cv2.rectangle(frame, (0,0), (400,73), (245,117,16), -1)
        cv2.putText(frame, stage, 
                    (20,50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
    if state == 1:
        cv2.putText(frame, stage, 
                    (20,50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)

    # Rendering 
    confidence_threshold = 0.3
    if state == 0:
        y, x, c = frame.shape
        shaped = np.squeeze(np.multiply(keypoints_with_scores, [y,x,1]))
        for edge, color in EDGES.items():
            p1, p2 = edge
            y1, x1, c1 = shaped[p1]
            y2, x2, c2 = shaped[p2]

            if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
                cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255,204,153), 2)
    elif state == 1:
        y, x, c = frame.shape
        shaped = np.squeeze(np.multiply(keypoints_with_scores, [y,x,1]))
        for edge, color in EDGES.items():
            p1, p2 = edge
            y1, x1, c1 = shaped[p1]
            y2, x2, c2 = shaped[p2]

            if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
                cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,255,255), 2)
    elif state == 2:
        y, x, c = frame.shape
        shaped = np.squeeze(np.multiply(keypoints_with_scores, [y,x,1]))
        for edge, color in EDGES.items():
            p1, p2 = edge
            y1, x1, c1 = shaped[p1]
            y2, x2, c2 = shaped[p2]

            if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
                cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,255,0), 2)
    else:
        y, x, c = frame.shape
        shaped = np.squeeze(np.multiply(keypoints_with_scores, [y,x,1]))
        for edge, color in EDGES.items():
            p1, p2 = edge
            y1, x1, c1 = shaped[p1]
            y2, x2, c2 = shaped[p2]

            if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
                cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)
    draw_keypoints(frame, keypoints_with_scores, 0.3)
    
    cv2.imshow('MoveNet Thunder', frame)
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

In [9]:
frame.shape

(480, 640, 3)